# colab을 이용한 Natural Language Processing(NLP) 실습

🎯 학습 목표 : colab 환경에서 NLP 모델 학습 코드를 실행하고 결과를 확인할 수 있다.

- 실습 재료

| 항목 | 상세 |
| ---- | ---- |
| 🗂️ 데이터 | 네이버 영화 리뷰 데이터셋 |
| 🤖 NLP 언어 모델 | ELECTRA (KoElectra Model) |
| 🏗️ NLP 학습 프레임워크 | torch |
| 🐍 프로그래밍 언어 | Python |
| 👩‍💻 프로그래밍 환경 | Colab |


- colab에서 코드 실행 방법은 다음 그림을 참조해주시기 바랍니다.

    ![](https://i.imgur.com/0GoFr7q.png)


# 1. HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드

본 실습에서 사용할 학습모델 관련 패키지(Huggingface transformers)와 네이버 영화 리뷰 데이터셋(NSMC)를 다운로드 합니다.

In [ ]:
!pip install transformers
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

다운로드 받은 train 데이터셋과 test 데이터셋의 형식 및 내용을 확인해봅니다.

In [ ]:
!head ratings_train.txt
!head ratings_test.txt

# 필요한 패키지 및 GPU 설정

학습에 필요한 패키지를 호출하고, CPU가 아닌 GPU를 사용하도록 설정합니다.

In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm
# GPU 사용
device = torch.device("cuda")

# 2. 데이터셋 전처리를 위한 NSMCDataset 클래스 만들기

이번 실습에서 사용되는 데이터는 감정분류가 레이블링된 네이버 영화 리뷰 데이터셋를 이용합니다. 이 데이터셋에는 결측치나 중복을 포함하기에 이를 적절히 처리해야 합니다.




In [ ]:
class NSMCDataset(Dataset):
  # 일부 값중에 NaN이 있음...
  def __init__(self, csv_file):
    self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0)
    # 중복제거
    self.dataset.drop_duplicates(subset=['document'], inplace=True)
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

    print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:3].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
train_dataset = NSMCDataset("ratings_train.txt")
test_dataset = NSMCDataset("ratings_test.txt")

# 3. 모델 다운로드

학습을 위한 ELECTRA 모델을 다운로드 합니다.

In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)

# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

### 모델 레이어 보기

다운로드 받은 모델의 레이어를 확인합니다.

In [ ]:
# 모델 레이어 보기
model

# 4. ELECTRA를 활용한 감정 분류 모델 훈련 및 평가

라이브러리를 설치했다면 이제 학습을 위한 설정을 진행합니다.

아래 코드를 통하여 감정 분류 작업을 위해 ELECTRA 모델을 미세 조정하는 방법을 배웁니다.


In [ ]:
epochs = 1
batch_size = 10

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

## 4.1 학습진행

설정이 완료되었으면, 학습을 진행합니다.

원래는 여러 학습 epoch를 진행해야 하지만, 실습을 간략하게 진행하기 위해 1 epoch와 total 100 batch만으로 학습을 수행합니다.

In [ ]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    if batches == 100:
        print(f"Batch {batches}")
        break

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)

  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

학습 과정에서 모델의 학습 데이터셋 loss와 정확도를 확인합니다.

In [ ]:
losses, accuracies

## 4.2 테스트 데이터셋 정확도 확인하기

학습된 모델의 테스트 데이터셋 정확도를 확인합니다.

In [ ]:
model.eval()

test_correct = 0
test_total = 0
batches = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)
  if batches == 100:
        print(f"Batch {batches}")
        break
  batches += 1


print("Accuracy:", test_correct.float() / test_total)

## 4.3 모델 저장하기

학습과 평가가 완료되었다면, 학습된 모델을 저장합니다.

In [ ]:
torch.save(model.state_dict(), "model.pt")

이상으로 본 실습을 마치도록 하겠습니다.

모두들 고생 많으셨습니다!

![](https://img.favpng.com/10/1/7/kakaotalk-kakao-friends-emoticon-sticker-png-favpng-mZm2vp0mk2Ce9aTUnBjC4s4DZ.jpg)

#참고


## 사용 Framework : Pytorch + HuggingFace
## 사용 Model : KoElectra Model
KoElectra-small 사용<br>
https://monologg.kr/2020/05/02/koelectra-part1/<br>
https://github.com/monologg/KoELECTRA

## Dataset
네이버 영화 리뷰 데이터셋<br>
https://github.com/e9t/nsmc

## References
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249
- https://heegyukim.medium.com/